In [2]:
                                                                                            #se_f = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/SE.rds"
#outdir = "/data/Mito_Trace/output/pipeline/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_3/concat/gff_A2_black/annotation_clones/umap_clones_overlay"
#se_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/clones/variants_init/knn/kparam_30/gff_A2_black/annotation_clones/SE.rds"
# se_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/annotation/gff_A2_black/mergedSamples/allSamples.integrated.rds" 
# cells_meta_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_mt_bestparams_af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0/clonalShift_method_mt_as_clones_dendro/cells_meta.tsv"
# outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_mt_bestparams_af.0.1_othaf.0.1_cov.10_othcov.10_ncells.10_othncells.0.25_mean.0/clonalShift_method_mt_as_clones_dendro/umap_overlay"

# se_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/anno_multiplex/gff_A2_black/SE.rds"
# cells_meta_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_3/clonalShift_method_dendro_bc/cells_meta.tsv"
# outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_3/clonalShift_method_dendro_bc/single"

se_f  = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/anno_multiplex/gff_A2_black/SE.rds"
cells_meta_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/cells_meta.tsv"
outdir = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/"
to_rename=FALSE

clone_order_f = "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/clones_ranked/cloneID_rank_ncells.txt"
top_clones = 10

conditions = "Input,Flt3l"

In [3]:
#library("Nebulosa")
library("Seurat")
#library("Seurat")
library(magrittr)
library(cowplot)
library(ggplot2)
library(Signac)
library(patchwork)
set.seed(1234)
library(magrittr)
library(cowplot)
library(metap)
library(dplyr)
library(future)

Registered S3 method overwritten by 'spatstat.core':
  method          from
  formula.glmmPQL MASS

Attaching SeuratObject

Attaching sp

Warning message:
“package ‘magrittr’ was built under R version 4.1.3”
Warning message:
“package ‘ggplot2’ was built under R version 4.1.3”

Attaching package: ‘patchwork’


The following object is masked from ‘package:cowplot’:

    align_plots


Warning message:
“package ‘dplyr’ was built under R version 4.1.3”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Warning message:
“package ‘future’ was built under R version 4.1.3”


In [4]:
shape_vals = c(16,1,15,0) #map for the shapes for the conditions in the umap
conditions_l = unlist(strsplit(conditions, ",",),)
conditions_l

shape_vals = shape_vals[1:(length(conditions_l))]
shape.map <- cbind(conditions_l, shape_vals)
shape.map

[1] "Input" "Flt3l"

conditions_l,shape_vals
Input,16
Flt3l,1


In [5]:
clone_order <- as.data.frame(readr::read_csv(clone_order_f,col_names=F))$X1
clone_order

top.clones <- clone_order[1:top_clones]
top.clones

Rows: 33 Columns: 1
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: ","
chr (1): X1

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "0_0"  "0_1"  "0_2"  "0_3"  "0_4"  "0_5"  "0_6"  "0_8"  "0_7"  "0_9" 
[11] "0_10" "0_11" "0_12" "0_13" "0_15" "0_16" "0_14" "0_17" "0_18" "0_19"
[21] "0_20" "0_21" "0_22" "0_23" "0_24" "0_25" "0_26" "0_27" "0_28" "0_29"
[31] "0_30" "0_31" "0_32"

[1] "0_0" "0_1" "0_2" "0_3" "0_4" "0_5" "0_6" "0_8" "0_7" "0_9"

In [6]:
cells_meta <- as.data.frame(readr::read_tsv(cells_meta_f),) # %>% tidyr::unite(barcode)

curr.cols <- colnames(cells_meta) 
curr.cols[[1]] = "cell"
curr.cols
colnames(cells_meta)  <- curr.cols
row.names(cells_meta) <- cells_meta$cell
cells_meta

New names:
• `` -> `...1`
Rows: 6696 Columns: 5
── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (4): ...1, cloneID, clusterID, condition
dbl (1): donor

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "cell"      "cloneID"   "clusterID" "condition" "donor"

,cell,cloneID,clusterID,condition,donor
,<chr>,<chr>,<chr>,<chr>,<dbl>
Flt3l_AAACGAACAAGCGAAC-1,Flt3l_AAACGAACAAGCGAAC-1,0_8,Granulocyte,Flt3l,0
Flt3l_AAACGAACACAGCCAC-1,Flt3l_AAACGAACACAGCCAC-1,0_27,Non-classical monocyte,Flt3l,0
Flt3l_AAACGAACACAGGTAG-1,Flt3l_AAACGAACACAGGTAG-1,0_8,Erythroid,Flt3l,0
Flt3l_AAACGAACATAGAATG-1,Flt3l_AAACGAACATAGAATG-1,0_8,Classical Monocyte,Flt3l,0
Flt3l_AAACGAAGTATTCGAC-1,Flt3l_AAACGAAGTATTCGAC-1,0_12,Non-classical monocyte,Flt3l,0
Flt3l_AAACGAAGTCTGATTG-1,Flt3l_AAACGAAGTCTGATTG-1,0_13,HSPC 3,Flt3l,0
Flt3l_AAACGAATCGCTGATA-1,Flt3l_AAACGAATCGCTGATA-1,0_19,Erythroid,Flt3l,0
Flt3l_AAACTCGAGATGTTCC-1,Flt3l_AAACTCGAGATGTTCC-1,0_1,Neutrophil,Flt3l,0
Flt3l_AAACTCGAGTCTCCAA-1,Flt3l_AAACTCGAGTCTCCAA-1,0_20,Neutrophil,Flt3l,0


## Need to change the cell IDs to have condition as prefix

In [7]:
se <- readRDS(se_f)
se$ID <- apply(stringr::str_split((rownames(se[[]])), "_", simplify=T)[,1:2], 1, function(x) {paste0(x[[2]],  "_", x[[1]])})
if(to_rename){
se <- RenameCells(se, new.names=se$ID)
}
coord = max(max(Embeddings(se[["umap"]])), abs(min(Embeddings(se[["umap"]]))))
coord
se[[]]



[1] 7.945776

,orig.ident,nCount_ATAC,nFeature_ATAC,total,duplicate,chimeric,unmapped,lowmapq,mitochondrial,passed_filters,⋯,nCount_RNA,nFeature_RNA,ID,Row.names,index,raw.ID,new.index,condition,donor,donor_index
,<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<chr>,<I<chr>>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<dbl>
Flt3l_AAACGAAAGGAGACTC-1,Flt3l,531,531,2286,298,7,7,124,749,1101,⋯,649,621,AAACGAAAGGAGACTC-1_Flt3l,Flt3l_AAACGAAAGGAGACTC-1,0,AAACGAAAGGAGACTC-1,1,Flt3l,1,1
Flt3l_AAACGAAAGTCACGCC-1,Flt3l,1222,1222,4711,555,16,12,289,853,2986,⋯,1674,1502,AAACGAAAGTCACGCC-1_Flt3l,Flt3l_AAACGAAAGTCACGCC-1,1,AAACGAAAGTCACGCC-1,2,Flt3l,1,2
Flt3l_AAACGAACAAGACTAA-1,Flt3l,2225,2225,9178,1509,37,19,585,1339,5689,⋯,3369,2695,AAACGAACAAGACTAA-1_Flt3l,Flt3l_AAACGAACAAGACTAA-1,2,AAACGAACAAGACTAA-1,3,Flt3l,1,3
Flt3l_AAACGAACAAGCGAAC-1,Flt3l,2271,2271,8825,1544,24,24,535,1308,5390,⋯,3360,2784,AAACGAACAAGCGAAC-1_Flt3l,Flt3l_AAACGAACAAGCGAAC-1,3,AAACGAACAAGCGAAC-1,4,Flt3l,0,1
Flt3l_AAACGAACACAGCCAC-1,Flt3l,2854,2854,10589,1294,41,31,456,2764,6003,⋯,3722,2998,AAACGAACACAGCCAC-1_Flt3l,Flt3l_AAACGAACACAGCCAC-1,4,AAACGAACACAGCCAC-1,5,Flt3l,0,2
Flt3l_AAACGAACACAGGTAG-1,Flt3l,615,615,2384,240,13,7,149,594,1381,⋯,901,865,AAACGAACACAGGTAG-1_Flt3l,Flt3l_AAACGAACACAGGTAG-1,5,AAACGAACACAGGTAG-1,6,Flt3l,0,3
Flt3l_AAACGAACACAGTAGG-1,Flt3l,2545,2545,10019,1869,31,28,514,1008,6569,⋯,3950,3101,AAACGAACACAGTAGG-1_Flt3l,Flt3l_AAACGAACACAGTAGG-1,6,AAACGAACACAGTAGG-1,7,Flt3l,1,4
Flt3l_AAACGAACATAGAATG-1,Flt3l,985,985,3395,433,8,20,163,985,1786,⋯,1090,1005,AAACGAACATAGAATG-1_Flt3l,Flt3l_AAACGAACATAGAATG-1,7,AAACGAACATAGAATG-1,8,Flt3l,0,4
Flt3l_AAACGAACATGTAGAA-1,Flt3l,1020,1020,5086,708,5,21,243,1777,2332,⋯,1347,1237,AAACGAACATGTAGAA-1_Flt3l,Flt3l_AAACGAACATGTAGAA-1,8,AAACGAACATGTAGAA-1,9,Flt3l,1,5


In [8]:
#se[[]][!duplicated(se[[]][ , c("seurat_clusters", "clusterID")]), ][c("seurat_clusters", "clusterID")]

## Filter for top clones

In [9]:
se <- AddMetaData(se, cells_meta)
se

An object of class Seurat 
212927 features across 13593 samples within 2 assays 
Active assay: RNA (19039 features, 0 variable features)
 1 other assay present: ATAC
 2 dimensional reductions calculated: integrated_lsi, umap

In [10]:
palette1 <- rainbow(length(unique(se$clusterID)))
palette1

f <- function(a,b){
    return(a=b)
}
colors <- mapply(f, unique(se$clusterID),palette1)
       #c('#F68282','#31C53F','#1FA195','#B95FBB','#D4D915','#28CECA','#ff9a36','#2FF18B','#aeadb3','#faf4cf','#CCB1F1','#25aff5','#A4DFF2','#4B4BF7','#AC8F14','#E6C122') )
colors

[1] "#FF0000" "#FF7600" "#FFEB00" "#9DFF00" "#27FF00" "#00FF4E" "#00FFC4"
 [8] "#00C4FF" "#004EFF" "#2700FF" "#9D00FF" "#FF00EB" "#FF0076"

<NA>            Granulocyte Non-classical monocyte 
             "#FF0000"              "#FF7600"              "#FFEB00" 
             Erythroid     Classical Monocyte                 HSPC 3 
             "#9DFF00"              "#27FF00"              "#00FF4E" 
            Neutrophil                 HPSC 2                 HSPC 4 
             "#00FFC4"              "#00C4FF"              "#004EFF" 
           pre-B cells                 HPSC 1                    MkP 
             "#2700FF"              "#9D00FF"              "#FF00EB" 
                   pDC 
             "#FF0076"

In [9]:
cloneID <- (unique(se$cloneID))
#cloneID <- (unique(se$cloneID))
cloneID <- cloneID[! cloneID=="None"]
cloneID <- cloneID[! cloneID=="None"]
cloneID <- na.omit(cloneID)
cloneID

cond <- (unique(cells_meta$condition))
cond <- cond[! cond=="None"]
cond

donors <- levels(as.factor(cells_meta$donor))
donors <-  donors[donors != "None"]
donors

[1] "0_8"  "0_27" "0_12" "0_13" "0_19" "0_1"  "0_20" "0_9"  "0_0"  "0_6" 
[11] "0_15" "0_30" "0_3"  "0_17" "0_5"  "0_2"  "0_26" "0_23" "0_16" "0_24"
[21] "0_14" "0_7"  "0_28" "0_29" "0_22" "0_25" "0_18" "0_10" "0_4"  "0_32"
[31] "0_31" "0_21" "0_11"
attr(,"na.action")
[1] 1
attr(,"class")
[1] "omit"

[1] "Flt3l" "Input"

[1] "0"

In [10]:
se <- subset(x = se, subset = (cloneID != "None") & (donor != "None") & (condition != "None"))
top.se <- subset(x = se, subset = cloneID %in% top.clones)
top.se$cloneID <- factor(x = top.se$cloneID, levels =top.clones)


In [ ]:

# my_cols <- c('3'='#F68282','15'='#31C53F','5'='#1FA195','1'='#B95FBB','13'='#D4D915',
#   '14'='#28CECA','9'='#ff9a36','8'='#2FF18B','11'='#aeadb3','6'='#faf4cf',
#   '2'='#CCB1F1','12'='#25aff5','7'='#A4DFF2','4'='#4B4BF7','16'='#AC8F14',
#   '10'='#E6C122')

# my_cols2 <- my_cols[order(as.integer(names(my_cols)))]
# scales::show_col(my_cols2)

In [ ]:
#se$nuclear_cluster

In [ ]:

# my_cols <- c('3'='#F68282','15'='#31C53F','5'='#1FA195','1'='#B95FBB','13'='#D4D915',
#   '14'='#28CECA','9'='#ff9a36','8'='#2FF18B','11'='#aeadb3','6'='#faf4cf',
#   '2'='#CCB1F1','12'='#25aff5','7'='#A4DFF2','4'='#4B4BF7','16'='#AC8F14',
#   '10'='#E6C122')

# my_cols2 <- my_cols[order(as.integer(names(my_cols)))]
# scales::show_col(my_cols2)


# DimPlot(my_seurat,
#         cols = my_cols2, label=TRUE , repel=TRUE)

In [29]:
# d <- DimPlot(object = top.se, label = FALSE, split.by="cloneID", order = top.clones, combine=T,
#              group.by=c("clusterID"), shape.by=c("condition"), pt.size=2) +
#     xlim(-1*coord-0.01,coord+0.01) +
#     ylim(-1*coord-0.01,coord+0.01)

d <- DimPlot(object = top.se, label = FALSE, split.by="cloneID", order = top.clones, ncol=1, 
             combine=T, cols=colors,
             group.by=c("clusterID"), shape.by=c("condition"), pt.size=10) +
    xlim(-1*coord-0.01,coord+0.01) +
    ylim(-1*coord-0.01,coord+0.01) + 
    theme(text = element_text(size = 4*top_clones)) +
    theme(axis.text = element_text(size = (top_clones))) +  
    scale_shape_manual(values = c("Input" = 16, "Flt3l" = 1)) + 
    scale_alpha_manual( values= c(.1,1))

ggsave(file.path(outdir, "top_umap_clust_col.svg"), width=2*top_clones, height=8*top_clones, device='svg', units="in", limitsize = FALSE)
ggsave(file.path(outdir, "top_umap_clust_col.pdf"), width=2*top_clones, height=8*top_clones, units="in", limitsize = FALSE)

In [28]:
outdir

[1] "/data/Mito_Trace/output/pipeline/v04/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/single_clones/donor0/cloneMethod_variants_init_knn_resolution_30/clonalShift_method_clones/top/"

In [ ]:
# d <- DimPlot(object = top.se, label = FALSE, split.by="cloneID", order = top.clones, ncol=1, 
#              combine=T,
#              group.by=c("condition"),  pt.size=2*(top_clones/2)) +
#     xlim(-1*coord-0.01,coord+0.01) +
#     ylim(-1*coord-0.01,coord+0.01) + 
#     theme(text = element_text(size = 8*top_clones)) +
#     theme(axis.text = element_text(size = 20))

# ggsave(file.path(outdir, "top_umap_cond_col.svg"), width=4*top_clones, height=4*top_clones*top_clones,limitsize = FALSE)
# ggsave(file.path(outdir, "top_umap_cond_col.pdf"), width=4*top_clones, height=4*top_clones*top_clones,limitsize = FALSE)



d <- DimPlot(object = top.se, label = FALSE, split.by="cloneID", order = top.clones, ncol=1, 
             combine=T, 
             group.by=c("condition"),  pt.size=12 ) +
    xlim(-1*coord-0.01,coord+0.01) +
    ylim(-1*coord-0.01,coord+0.01) + 
    theme(text = element_text(size = 4*top_clones)) +
    theme(axis.text = element_text(size = (top_clones)))

ggsave(file.path(outdir, "top_umap_cond_col.svg"), width=2*top_clones, height=8*top_clones, device='svg', units="in", limitsize = FALSE)
ggsave(file.path(outdir, "top_umap_cond_col.pdf"), width=2*top_clones, height=8*top_clones, units="in", limitsize = FALSE)

In [ ]:
#RidgePlot(object = se, group.by=cloneID, features = c('UMAP_1', 'UMAP_2'), sort=F)